In [ ]:
!pip install --quiet transformers evaluate datasets baal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install --quiet --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/RuSportSum/train_selector_after_match2.csv" /content/train.csv
!cp "/content/drive/MyDrive/RuSportSum/eval_selector_after_match2.csv" /content/eval.csv
!cp -r "/content/drive/MyDrive/RuSportSum/bald" "/content/bald"

In [ ]:
import os
import random
from copy import deepcopy

import pandas as pd
import numpy as np
import torch.backends
import transformers
import evaluate

from datasets import load_dataset
from tqdm import tqdm
from transformers import BertForSequenceClassification
from transformers import BertTokenizer, TrainingArguments
from transformers import set_seed

from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    set_seed,
    Trainer,
    TrainingArguments
)

transformers.utils.logging.set_verbosity_warning()

from baal.active import get_heuristic
from baal.active.dataset.nlp_datasets import (
    active_huggingface_dataset,
    HuggingFaceDatasets,
)
from baal.bayesian.dropout import patch_module
from baal.transformers_trainer_wrapper import BaalTransformersTrainer
from baal.active import get_heuristic, ActiveLearningDataset
from baal.active.dataset.base import Dataset

from typing import List
from pprint import pprint

SEED = 3

random.seed(SEED)
torch.manual_seed(SEED)

# Set tranformer seed to ensure that initial weights are identical
set_seed(SEED)

In [ ]:
train_data = pd.read_csv("/content/train.csv")
eval_data = pd.read_csv("/content/eval.csv")

In [ ]:
train_data

,match_id,news_id,name,team1,team2,relevant_part,type,minute,content,news_name_body
0,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,NaN,0,"0 минута. Салют амигос! ""Барса"" уже в трех очк...",Серхио Рамос одержал 150-ю победу с «Реалом» в...
1,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,NaN,0,"0 минута. Без Пепе, без Озила и без Диарра. И ...",Серхио Рамос одержал 150-ю победу с «Реалом» в...
2,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,NaN,1,1 минута. Поехали!,Серхио Рамос одержал 150-ю победу с «Реалом» в...
3,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,NaN,2,2 минута. Первый удар за гостями - вовремя под...,Серхио Рамос одержал 150-ю победу с «Реалом» в...
4,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,NaN,5,"5 минута. ""Реал"" начал раскачивать. А Бензема ...",Серхио Рамос одержал 150-ю победу с «Реалом» в...
...,...,...,...,...,...,...,...,...,...,...
748696,475786,140849366,Чемпионат мира,Канада,Словакия,0,empty,60,60 минута. В последний раз так далеко словаки ...,Канада в третий раз подряд не сумела преодолет...
748697,475786,140849366,Чемпионат мира,Канада,Словакия,0,empty,60,"60 минута. Канада же в третий, повторяю, в тре...",Канада в третий раз подряд не сумела преодолет...
748698,475786,140849366,Чемпионат мира,Канада,Словакия,0,empty,60,60 минута. Гецлафу из Хельсинки лучше лететь с...,Канада в третий раз подряд не сумела преодолет...
748699,475786,140849366,Чемпионат мира,Канада,Словакия,0,empty,60,60 минута. А наша трансляция подошла к концу. ...,Канада в третий раз подряд не сумела преодолет...


In [ ]:
data_for_predict = train_data.loc[21188:]
data_for_predict.to_csv('data_for_predict.csv', index=False, encoding='utf-8')

In [ ]:
load_data = load_dataset("csv", data_files={'train': '/content/data_for_predict.csv'})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-bd12be57c49da957/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

model = AutoModelForSequenceClassification.from_pretrained("/content/bald")

training_args = TrainingArguments(
    output_dir=".",
    num_train_epochs=2,
    per_device_train_batch_size=48,
    per_device_eval_batch_size=48,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = BaalTransformersTrainer(
    model=model,
    args=training_args,
    tokenizer=None
)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['content'], examples['news_name_body'], max_length=200, truncation=True, padding="max_length")

In [ ]:
tokenized_data = load_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/727513 [00:00<?, ? examples/s]

In [ ]:
output = trainer.predict(tokenized_data['train'])

In [ ]:
predict_labels = np.argmax(output.predictions, axis=1)

In [ ]:
data_for_predict['relevant_part'] = predict_labels
data_for_predict

<ipython-input-21-dcc4896a788a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_predict['relevant_part'] = predict_labels


,match_id,news_id,name,team1,team2,relevant_part,type,minute,content,news_name_body
21188,1021128,1028108673,Кубок Гагарина,СКА,Динамо Москва,0,empty,11,"11 минута. ""Динамо"" надолго обустраивается в з...",КХЛ. СКА вышел в финал Западной конференции & ...
21189,1021128,1028108673,Кубок Гагарина,СКА,Динамо Москва,0,empty,12,"12 минута. Даже трибуны немного притихли, СКА ...",КХЛ. СКА вышел в финал Западной конференции & ...
21190,1021128,1028108673,Кубок Гагарина,СКА,Динамо Москва,0,empty,12,"12 минута. Ковальчук получил повреждение уха, ...",КХЛ. СКА вышел в финал Западной конференции & ...
21191,1021128,1028108673,Кубок Гагарина,СКА,Динамо Москва,0,empty,13,13 минута. Активизировались динамовцы в броско...,КХЛ. СКА вышел в финал Западной конференции & ...
21192,1021128,1028108673,Кубок Гагарина,СКА,Динамо Москва,0,empty,14,"14 минута. СКА защищается глубоко, силой четве...",КХЛ. СКА вышел в финал Западной конференции & ...
...,...,...,...,...,...,...,...,...,...,...
748696,475786,140849366,Чемпионат мира,Канада,Словакия,0,empty,60,60 минута. В последний раз так далеко словаки ...,Канада в третий раз подряд не сумела преодолет...
748697,475786,140849366,Чемпионат мира,Канада,Словакия,0,empty,60,"60 минута. Канада же в третий, повторяю, в тре...",Канада в третий раз подряд не сумела преодолет...
748698,475786,140849366,Чемпионат мира,Канада,Словакия,0,empty,60,60 минута. Гецлафу из Хельсинки лучше лететь с...,Канада в третий раз подряд не сумела преодолет...
748699,475786,140849366,Чемпионат мира,Канада,Словакия,0,empty,60,60 минута. А наша трансляция подошла к концу. ...,Канада в третий раз подряд не сумела преодолет...


In [ ]:
result_data = pd.concat([train_data.loc[:21188], data_for_predict, eval_data])
result_data

,match_id,news_id,name,team1,team2,relevant_part,type,minute,content,news_name_body
0,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,NaN,0,"0 минута. Салют амигос! ""Барса"" уже в трех очк...",Серхио Рамос одержал 150-ю победу с «Реалом» в...
1,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,NaN,0,"0 минута. Без Пепе, без Озила и без Диарра. И ...",Серхио Рамос одержал 150-ю победу с «Реалом» в...
2,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,NaN,1,1 минута. Поехали!,Серхио Рамос одержал 150-ю победу с «Реалом» в...
3,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,NaN,2,2 минута. Первый удар за гостями - вовремя под...,Серхио Рамос одержал 150-ю победу с «Реалом» в...
4,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,NaN,5,"5 минута. ""Реал"" начал раскачивать. А Бензема ...",Серхио Рамос одержал 150-ю победу с «Реалом» в...
...,...,...,...,...,...,...,...,...,...,...
8755,829996,1021343013,Чемпионат мира,Англия,Италия,0,empty,90,90 минута. Кэйхилл напоследок в идеальном подк...,Англия – Италия – 1:2. Балотелли забил победны...
8756,829996,1021343013,Чемпионат мира,Англия,Италия,0,empty,90,90 минута. Сборная Италии стартует на чемпиона...,Англия – Италия – 1:2. Балотелли забил победны...
8757,829996,1021343013,Чемпионат мира,Англия,Италия,0,empty,90,90 минута. Итоговая статистика: удары (в створ...,Англия – Италия – 1:2. Балотелли забил победны...
8758,829996,1021343013,Чемпионат мира,Англия,Италия,0,empty,90,"90 минута. Назвать матч, который мы посмотрели...",Англия – Италия – 1:2. Балотелли забил победны...


In [ ]:
result_data['relevant_part'].value_counts()

0    714792
1     42670
Name: relevant_part, dtype: int64

In [ ]:
result_data.to_csv('summ_after_match.csv', index=False, encoding='utf-8')

In [ ]:
!cp "/content/summ_after_match.csv" "/content/drive/MyDrive/RuSportSum/summ_after_match.csv"